In [1]:
import time
import pandas as pd
from datetime import timedelta
from growwapi import GrowwAPI

# --- CREDENTIALS ---
API_KEY = "eyJraWQiOiJaTUtjVXciLCJhbGciOiJFUzI1NiJ9.eyJleHAiOjI1NTM1Njg0NzYsImlhdCI6MTc2NTE2ODQ3NiwibmJmIjoxNzY1MTY4NDc2LCJzdWIiOiJ7XCJ0b2tlblJlZklkXCI6XCJhMTg3NDVhMy1hN2M1LTRlOTQtODE1MS1lZjUxZDQ5OGE2Y2RcIixcInZlbmRvckludGVncmF0aW9uS2V5XCI6XCJlMzFmZjIzYjA4NmI0MDZjODg3NGIyZjZkODQ5NTMxM1wiLFwidXNlckFjY291bnRJZFwiOlwiMDdmMDA0MGMtZTk4Zi00ZDNmLTk5Y2EtZDc1ZjBlYWU5M2NlXCIsXCJkZXZpY2VJZFwiOlwiZDMyMWIxMzUtZWQ5Mi01ZWJkLWJjMDUtZTY1NDY2OWRiMDM5XCIsXCJzZXNzaW9uSWRcIjpcIjBlOWMyYWZmLTM0NzktNDUyMi1iODE4LTczNTZlMzFkYmY1Y1wiLFwiYWRkaXRpb25hbERhdGFcIjpcIno1NC9NZzltdjE2WXdmb0gvS0EwYk1yOE5XVzhzdTNvZ080am1ZUzIwZEpSTkczdTlLa2pWZDNoWjU1ZStNZERhWXBOVi9UOUxIRmtQejFFQisybTdRPT1cIixcInJvbGVcIjpcImF1dGgtdG90cFwiLFwic291cmNlSXBBZGRyZXNzXCI6XCIyNDA5OjQwOTA6MTA4ZjpkYzA1OjNkMWQ6MWZmMDo1YWFjOjYwNTYsMTcyLjcxLjE5OC4xOSwzNS4yNDEuMjMuMTIzXCIsXCJ0d29GYUV4cGlyeVRzXCI6MjU1MzU2ODQ3NjQzNn0iLCJpc3MiOiJhcGV4LWF1dGgtcHJvZC1hcHAifQ.VuAMgqoC3e32gduObByNz97jFfG-ikXoREum26XPkvyMpj9JgCedXBI81jxGTPTrZD9i1wIL0s38LPd9vc9ApA"
API_SECRET = "xy0sbQ4r*!HN3&&UKc9vpwti4xx8PR)("

def auth():
    try:
        token = GrowwAPI.get_access_token(api_key=API_KEY, secret=API_SECRET)
        return GrowwAPI(token)
    except Exception as e:
        print(f"Auth failed: {e}")
        exit()

groww = auth()
print("Logged into Groww!")

Ready to Groww!
Logged into Groww!


In [2]:
import math

# Get a list of all attributes in the 'math' module
math_attributes = dir(GrowwAPI)

# Print each attribute
for attribute in math_attributes:
    print(attribute)

CANDLE_INTERVAL_DAY
CANDLE_INTERVAL_HOUR_1
CANDLE_INTERVAL_HOUR_4
CANDLE_INTERVAL_MIN_1
CANDLE_INTERVAL_MIN_10
CANDLE_INTERVAL_MIN_15
CANDLE_INTERVAL_MIN_2
CANDLE_INTERVAL_MIN_3
CANDLE_INTERVAL_MIN_30
CANDLE_INTERVAL_MIN_5
CANDLE_INTERVAL_MONTH
CANDLE_INTERVAL_WEEK
EXCHANGE_BSE
EXCHANGE_MCX
EXCHANGE_MCXSX
EXCHANGE_NCDEX
EXCHANGE_NSE
EXCHANGE_US
INSTRUMENT_CSV_URL
ORDER_TYPE_LIMIT
ORDER_TYPE_MARKET
ORDER_TYPE_STOP_LOSS
ORDER_TYPE_STOP_LOSS_MARKET
PRODUCT_ARBITRAGE
PRODUCT_BO
PRODUCT_CNC
PRODUCT_CO
PRODUCT_MIS
PRODUCT_MTF
PRODUCT_NRML
SEGMENT_CASH
SEGMENT_COMMODITY
SEGMENT_CURRENCY
SEGMENT_FNO
SMART_ORDER_STATUS_ACTIVE
SMART_ORDER_STATUS_CANCELLED
SMART_ORDER_STATUS_COMPLETED
SMART_ORDER_STATUS_EXPIRED
SMART_ORDER_STATUS_FAILED
SMART_ORDER_STATUS_TRIGGERED
SMART_ORDER_TYPE_GTT
SMART_ORDER_TYPE_OCO
TRANSACTION_TYPE_BUY
TRANSACTION_TYPE_SELL
TRIGGER_DIRECTION_DOWN
TRIGGER_DIRECTION_UP
VALIDITY_DAY
VALIDITY_EOS
VALIDITY_GTC
VALIDITY_GTD
VALIDITY_IOC
_ERROR_MAP
_GROWW_GENERATE_SOCKET_TOKEN_U

In [ ]:
# --- CONFIGURATION ---
# The new method requires these specific strings
SYMBOL = "NSE-NIFTY" 
EXCHANGE = "NSE"         
SEGMENT = "CASH"       
INTERVAL = "1minute"           
# --- CHUNKING LOGIC ---
total_days = 3994
chunk_size_days = 5
end_date = pd.Timestamp.now()
start_date = end_date - timedelta(days=total_days)

all_candles = []
current_start = start_date

In [ ]:


print(f"Starting batched download for {SYMBOL} ({SEGMENT})...")

while current_start < end_date:
    current_end = current_start + timedelta(days=chunk_size_days)
    if current_end > end_date:
        current_end = end_date
    
    # Format dates to string as required by signature
    s_str = current_start.strftime("%Y-%m-%d %H:%M:%S")
    e_str = current_end.strftime("%Y-%m-%d %H:%M:%S")
    
    print(f"Fetching: {s_str} -> {e_str}")
    
    try:
        # UPDATED CALL MATCHING YOUR SIGNATURE
        resp = groww.get_historical_candles(
            exchange=EXCHANGE,
            segment=SEGMENT,
            groww_symbol=SYMBOL,
            start_time=s_str,
            end_time=e_str,
            candle_interval=INTERVAL
        )
        
        # Parse response
        if resp and "candles" in resp and resp["candles"]:
            chunk_data = resp["candles"]
            all_candles.extend(chunk_data)
            print(f"  > Success: {len(chunk_data)} candles.")
        else:
            # Sometimes API returns empty or wrapped data, print to debug if needed
            print(f"  > No data/Empty response. (Check if Market was open)")
            
    except Exception as e:
        print(f"  > Error: {e}")

    current_start = current_end
    time.sleep(0.5) # Rate limit protection

# --- SAVE TO CSV ---
# --- SAVE TO CSV ---
if all_candles:
    # 1. Create DataFrame
    df = pd.DataFrame(all_candles)
    
    # 2. Rename Columns (Handle optional 7th column)
    base_cols = ["timestamp", "open", "high", "low", "close", "volume"]
    rename_map = {i: col for i, col in enumerate(base_cols)}
    df.rename(columns=rename_map, inplace=True)
    
    if len(df.columns) > 6:
        df.rename(columns={6: "open_interest"}, inplace=True)
    
    # 3. ROBUST TIMESTAMP FIX
    # Check the first value to decide how to convert
    first_ts = df["timestamp"].iloc[0]
    
    if isinstance(first_ts, str):
        # Case A: API returns strings like '2025-11-07T13:43:00'
        # Do NOT use unit='s' here.
        df["timestamp"] = pd.to_datetime(df["timestamp"])
    else:
        # Case B: API returns numbers (Epoch seconds) like 1735660000
        # MUST use unit='s' here.
        df["timestamp"] = pd.to_datetime(df["timestamp"], unit='s')
    
    # 4. Clean & Save
    df.sort_values("timestamp", inplace=True)
    df.drop_duplicates(subset="timestamp", inplace=True)
    
    filename = "nifty_spot_1m.csv"
    df.to_csv(filename, index=False)
    print(f"\nDONE! Saved {len(df)} candles to {filename}")
    print("First 5 rows:")
    print(df.head())
else:
    print("\nFAILED: No data collected.")

Starting batched download for NSE-NIFTY (CASH)...
Fetching: 2015-01-01 12:17:12 -> 2015-01-06 12:17:12
  > No data/Empty response. (Check if Market was open)
Fetching: 2015-01-06 12:17:12 -> 2015-01-11 12:17:12
  > No data/Empty response. (Check if Market was open)
Fetching: 2015-01-11 12:17:12 -> 2015-01-16 12:17:12
  > No data/Empty response. (Check if Market was open)
Fetching: 2015-01-16 12:17:12 -> 2015-01-21 12:17:12
  > No data/Empty response. (Check if Market was open)
Fetching: 2015-01-21 12:17:12 -> 2015-01-26 12:17:12
  > No data/Empty response. (Check if Market was open)
Fetching: 2015-01-26 12:17:12 -> 2015-01-31 12:17:12
  > No data/Empty response. (Check if Market was open)
Fetching: 2015-01-31 12:17:12 -> 2015-02-05 12:17:12
  > No data/Empty response. (Check if Market was open)
Fetching: 2015-02-05 12:17:12 -> 2015-02-10 12:17:12
  > No data/Empty response. (Check if Market was open)
Fetching: 2015-02-10 12:17:12 -> 2015-02-15 12:17:12
  > No data/Empty response. (Chec

In [6]:
INTERVALS = ["5minute","10minute","15minute","60minute","1day","1week"]

for INTERVAL in INTERVALS:
    all_candles = []
    current_start = start_date
    print("Downloading interval:", INTERVAL)
    while current_start < end_date:
        current_end = current_start + timedelta(days=chunk_size_days)
        if current_end > end_date:
            current_end = end_date
        s_str = current_start.strftime("%Y-%m-%d %H:%M:%S")
        e_str = current_end.strftime("%Y-%m-%d %H:%M:%S")
        try:
            resp = groww.get_historical_candles(
                exchange=EXCHANGE,
                segment=SEGMENT,
                groww_symbol=SYMBOL,
                start_time=s_str,
                end_time=e_str,
                candle_interval=INTERVAL
            )
            if resp and "candles" in resp and resp["candles"]:
                all_candles.extend(resp["candles"])
        except Exception as e:
            print("Error:", e)
        current_start = current_end
        time.sleep(0.5)

    if all_candles:
        df = pd.DataFrame(all_candles)
        # same robust timestamp fix as you already have...
        # save file name by interval
        df.to_csv(f"nifty_spot_{INTERVAL}.csv", index=False)
        print("Saved", f"nifty_spot_{INTERVAL}.csv")


Saved nifty_spot_5minute.csv
Saved nifty_spot_10minute.csv
Saved nifty_spot_15minute.csv
Error: Not able to recognize candle_interval, having value 60minute
Error: Not able to recognize candle_interval, having value 60minute
Error: Not able to recognize candle_interval, having value 60minute
Error: Not able to recognize candle_interval, having value 60minute
Error: Not able to recognize candle_interval, having value 60minute
Error: Not able to recognize candle_interval, having value 60minute
Error: Not able to recognize candle_interval, having value 60minute
Error: Not able to recognize candle_interval, having value 60minute
Error: Not able to recognize candle_interval, having value 60minute
Error: Not able to recognize candle_interval, having value 60minute
Error: Not able to recognize candle_interval, having value 60minute
Error: Not able to recognize candle_interval, having value 60minute
Error: Not able to recognize candle_interval, having value 60minute
Error: Not able to recogniz

In [1]:
import pandas as pd
from growwapi import GrowwAPI

# --- CREDENTIALS ---
API_KEY = "eyJraWQiOiJaTUtjVXciLCJhbGciOiJFUzI1NiJ9.eyJleHAiOjI1NTQzNTIwMzYsImlhdCI6MTc2NTk1MjAzNiwibmJmIjoxNzY1OTUyMDM2LCJzdWIiOiJ7XCJ0b2tlblJlZklkXCI6XCI3NzljMTAyNy03ZDQ1LTRlOWItYWM5ZS1iNDgzMWRiODQzZTFcIixcInZlbmRvckludGVncmF0aW9uS2V5XCI6XCJlMzFmZjIzYjA4NmI0MDZjODg3NGIyZjZkODQ5NTMxM1wiLFwidXNlckFjY291bnRJZFwiOlwiMDdmMDA0MGMtZTk4Zi00ZDNmLTk5Y2EtZDc1ZjBlYWU5M2NlXCIsXCJkZXZpY2VJZFwiOlwiZDMyMWIxMzUtZWQ5Mi01ZWJkLWJjMDUtZTY1NDY2OWRiMDM5XCIsXCJzZXNzaW9uSWRcIjpcIjllODBhNjM2LTY4OGMtNDQ4OC1hMDhjLTU1NzQwMDQwNDMwZlwiLFwiYWRkaXRpb25hbERhdGFcIjpcIno1NC9NZzltdjE2WXdmb0gvS0EwYk1yOE5XVzhzdTNvZ080am1ZUzIwZEpSTkczdTlLa2pWZDNoWjU1ZStNZERhWXBOVi9UOUxIRmtQejFFQisybTdRPT1cIixcInJvbGVcIjpcImF1dGgtdG90cFwiLFwic291cmNlSXBBZGRyZXNzXCI6XCIyNDA5OjQwOTA6MTA4ZjpkYzA1OmM5ODU6OWEzNjo2ZTEyOjFjZWIsMTYyLjE1OC4yMzUuMjA0LDM1LjI0MS4yMy4xMjNcIixcInR3b0ZhRXhwaXJ5VHNcIjoyNTU0MzUyMDM2MzA0fSIsImlzcyI6ImFwZXgtYXV0aC1wcm9kLWFwcCJ9.vFYYnOrSLi-teVY6qhFF11SeSVZRIo-xBz_lVlOoTDujYw3ucWZSbOoP9sqFg11Oc8cCwWASqbg_R-9BfmPU0Q"
API_SECRET = "Gb!4#@-d4*XbNz)F!)Y0iW8122uJiaTn"

# --- CONFIGURATION ---
SYMBOL = "NSE-NIFTY-23Dec25-25800-CE" 
START_TIME = "2025-12-17 09:15:00"
END_TIME   = "2025-12-17 14:50:00"
OUTPUT_CSV = "single_contract_2025.csv"

def main():
    try:
        print("1. Authenticating...")
        token = GrowwAPI.get_access_token(api_key=API_KEY, secret=API_SECRET)
        groww = GrowwAPI(token)
        
        print(f"2. Fetching {SYMBOL}...")
        
        resp = groww.get_historical_candles(
            exchange="NSE",
            segment="FNO",
            groww_symbol=SYMBOL,
            start_time=START_TIME,
            end_time=END_TIME,
            candle_interval="1minute"
        )
        
        if resp and "candles" in resp and len(resp["candles"]) > 0:
            print(f"3. Success! Data found ({len(resp['candles'])} rows).")
            
            # Create DataFrame
            df = pd.DataFrame(resp["candles"])
            
            # --- FIX 1: Handle 7 Columns (Auto-Detect OI) ---
            if len(df.columns) == 7:
                df.columns = ['ts', 'o', 'h', 'l', 'close', 'v', 'oi']
            elif len(df.columns) == 6:
                df.columns = ['ts', 'o', 'h', 'l', 'close', 'v']
            else:
                print(f"Warning: Unexpected column count {len(df.columns)}")
                df.rename(columns={0: 'ts', 4: 'close'}, inplace=True)
            
            # --- FIX 2: Handle String vs Number Timestamp ---
            # Check the first value to see if it's a string ('2020-01-01...') or int (1577836800)
            first_ts = df['ts'].iloc[0]
            
            if isinstance(first_ts, str):
                # It's already a string, pandas parses it directly
                df['timestamp'] = pd.to_datetime(df['ts'])
            else:
                # It's a number (Unix timestamp), need unit='s'
                df['timestamp'] = pd.to_datetime(df['ts'], unit='s')
            
            # Clean up
            final_cols = ['timestamp', 'o', 'h', 'l', 'close', 'v']
            if 'oi' in df.columns:
                final_cols.append('oi')
                
            df = df[final_cols]
            
            print("\n--- FIRST 5 ROWS ---")
            print(df.head())
            
            df.to_csv(OUTPUT_CSV, index=False)
            print(f"\n✅ Saved to {OUTPUT_CSV}")
            
        else:
            print("❌ No data returned.")

    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()

1. Authenticating...
Ready to Groww!
2. Fetching NSE-NIFTY-23Dec25-25800-CE...
3. Success! Data found (336 rows).

--- FIRST 5 ROWS ---
            timestamp       o       h      l   close       v
0 2025-12-17 09:15:00  182.50  189.70  172.8  182.00  585075
1 2025-12-17 09:16:00  184.15  197.90  182.2  197.90  734325
2 2025-12-17 09:17:00  198.20  200.05  194.3  197.80  581700
3 2025-12-17 09:18:00  197.05  199.25  192.6  196.35  357150
4 2025-12-17 09:19:00  196.85  197.40  191.7  192.95  237975

✅ Saved to single_contract_2025.csv


In [11]:
import pandas as pd
import time
from datetime import timedelta
from growwapi import GrowwAPI

# --- CREDENTIALS ---
API_KEY = "eyJraWQiOiJaTUtjVXciLCJhbGciOiJFUzI1NiJ9.eyJleHAiOjI1NTM1Njg0NzYsImlhdCI6MTc2NTE2ODQ3NiwibmJmIjoxNzY1MTY4NDc2LCJzdWIiOiJ7XCJ0b2tlblJlZklkXCI6XCJhMTg3NDVhMy1hN2M1LTRlOTQtODE1MS1lZjUxZDQ5OGE2Y2RcIixcInZlbmRvckludGVncmF0aW9uS2V5XCI6XCJlMzFmZjIzYjA4NmI0MDZjODg3NGIyZjZkODQ5NTMxM1wiLFwidXNlckFjY291bnRJZFwiOlwiMDdmMDA0MGMtZTk4Zi00ZDNmLTk5Y2EtZDc1ZjBlYWU5M2NlXCIsXCJkZXZpY2VJZFwiOlwiZDMyMWIxMzUtZWQ5Mi01ZWJkLWJjMDUtZTY1NDY2OWRiMDM5XCIsXCJzZXNzaW9uSWRcIjpcIjBlOWMyYWZmLTM0NzktNDUyMi1iODE4LTczNTZlMzFkYmY1Y1wiLFwiYWRkaXRpb25hbERhdGFcIjpcIno1NC9NZzltdjE2WXdmb0gvS0EwYk1yOE5XVzhzdTNvZ080am1ZUzIwZEpSTkczdTlLa2pWZDNoWjU1ZStNZERhWXBOVi9UOUxIRmtQejFFQisybTdRPT1cIixcInJvbGVcIjpcImF1dGgtdG90cFwiLFwic291cmNlSXBBZGRyZXNzXCI6XCIyNDA5OjQwOTA6MTA4ZjpkYzA1OjNkMWQ6MWZmMDo1YWFjOjYwNTYsMTcyLjcxLjE5OC4xOSwzNS4yNDEuMjMuMTIzXCIsXCJ0d29GYUV4cGlyeVRzXCI6MjU1MzU2ODQ3NjQzNn0iLCJpc3MiOiJhcGV4LWF1dGgtcHJvZC1hcHAifQ.VuAMgqoC3e32gduObByNz97jFfG-ikXoREum26XPkvyMpj9JgCedXBI81jxGTPTrZD9i1wIL0s38LPd9vc9ApA"
API_SECRET = "xy0sbQ4r*!HN3&&UKc9vpwti4xx8PR)("

# --- CONFIGURATION ---
SPOT_FILE = "Final_nifty_spot_1minute_fixed_trimmed.csv" # Your uploaded file
OUTPUT_FILE = "test_pilot_output.csv"

def auth():
    try:
        token = GrowwAPI.get_access_token(api_key=API_KEY, secret=API_SECRET)
        return GrowwAPI(token)
    except Exception as e:
        print(f"CRITICAL: Auth failed. Error: {e}")
        exit()

# --- 1. FIXED EXPIRY LOGIC (Handles Timezones & 2025 Switch) ---
def get_target_expiry_date(date_obj):
    # CRITICAL FIX: Remove timezone info (e.g. +05:30) for safe comparison
    date_obj = date_obj.replace(tzinfo=None)
    
    # Logic: Thursday before Sep 1, 2025. Tuesday after.
    switch_date = pd.Timestamp("2025-09-01")
    target_day = 1 if date_obj >= switch_date else 3 # 1=Tue, 3=Thu
    
    days_ahead = target_day - date_obj.weekday()
    if days_ahead < 0: 
        days_ahead += 7
        
    return date_obj + timedelta(days=days_ahead)

def run_pilot_test():
    # 1. Load Data
    print(f"1. Loading Spot Data from {SPOT_FILE}...")
    try:
        df_spot = pd.read_csv(SPOT_FILE)
        
        # Handle Timestamps with Timezone (e.g. +05:30)
        # We use utc=True to normalize, then convert to naive if needed, or just let pandas handle it
        df_spot['timestamp'] = pd.to_datetime(df_spot['timestamp'])
        
        # Remove timezone for easier processing
        df_spot['timestamp'] = df_spot['timestamp'].dt.tz_localize(None)
        
        df_spot.sort_values('timestamp', inplace=True)
        print(f"   Loaded {len(df_spot)} rows.")
        
    except FileNotFoundError:
        print("   Error: File not found.")
        return

    # 2. Select Test Ranges (First 2 days of 2020 and Last 2 days of data)
    # This proves both 'Old' and 'New' formats work
    start_2020 = df_spot['timestamp'].min()
    end_2020 = start_2020 + timedelta(days=3)
    
    start_recent = df_spot['timestamp'].max() - timedelta(days=3)
    end_recent = df_spot['timestamp'].max()
    
    mask = ((df_spot['timestamp'] >= start_2020) & (df_spot['timestamp'] <= end_2020)) | \
           ((df_spot['timestamp'] >= start_recent) & (df_spot['timestamp'] <= end_recent))
           
    df_test = df_spot.loc[mask].copy()
    print(f"2. Pilot Subset: {len(df_test)} rows (Jan 2020 & Recent).")

    # 3. Prepare Data
    df_test['atm_strike'] = (df_test['close'] / 50).round() * 50
    df_test['expiry_dt'] = df_test['timestamp'].apply(get_target_expiry_date)
    df_test['expiry_str'] = df_test['expiry_dt'].dt.strftime("%d%b%y")
    
    # 4. Run Download Loop (Mini Version)
    groww = auth()
    option_cache = {}
    grouped = df_test.groupby('expiry_str')
    
    print("\n--- Starting Pilot Download ---")
    
    rows_collected = []
    
    for expiry_str, group in grouped:
        print(f"\nProcessing Expiry: {expiry_str}")
        
        # Identify Strikes
        atm_strikes = group['atm_strike'].unique().astype(int)
        strikes_to_fetch = set()
        for k in atm_strikes:
            strikes_to_fetch.add(k)      # ATM
            strikes_to_fetch.add(k - 50) # ITM
            strikes_to_fetch.add(k + 50) # OTM
        
        # Date Logic
        primary_dt = group['expiry_dt'].iloc[0]
        backup_dt = primary_dt - timedelta(days=1)
        fmt_primary = primary_dt.strftime("%d%b%y")
        fmt_backup = backup_dt.strftime("%d%b%y")
        
        # Active Format Detection
        active_fmt = fmt_primary
        # Quick check using first available strike
        test_sym = f"NSE-NIFTY-{fmt_primary}-{list(strikes_to_fetch)[0]}-CE"
        
        try:
            check_start = group['timestamp'].min().strftime("%Y-%m-%d %H:%M:%S")
            check_end = (group['timestamp'].min() + timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")
            resp = groww.get_historical_candles(
                exchange="NSE", segment="FNO", groww_symbol=test_sym,
                start_time=check_start, end_time=check_end, candle_interval="1minute"
            )
            if not (resp and "candles" in resp and len(resp["candles"]) > 0):
                print(f"   Note: Primary expiry {fmt_primary} empty, checking backup...")
                active_fmt = fmt_backup
        except:
            active_fmt = fmt_backup
            
        print(f"   Using Date: {active_fmt} | Fetching {len(strikes_to_fetch)*2} contracts...")

        # Download
        start_dt_str = group['timestamp'].min().strftime("%Y-%m-%d %H:%M:%S")
        end_dt_str = group['timestamp'].max().strftime("%Y-%m-%d %H:%M:%S")
        
        for strike in strikes_to_fetch:
            for opt_type in ['CE', 'PE']:
                sym = f"NSE-NIFTY-{active_fmt}-{strike}-{opt_type}"
                if sym in option_cache: continue
                
                try:
                    resp = groww.get_historical_candles(
                        exchange="NSE", segment="FNO", groww_symbol=sym,
                        start_time=start_dt_str, end_time=end_dt_str, candle_interval="1minute"
                    )
                    
                    if resp and "candles" in resp and len(resp["candles"]) > 0:
                        df = pd.DataFrame(resp["candles"])
                        
                        # FIX: Handle Column Variations
                        if len(df.columns) == 7:
                            df.columns = ['ts', 'o', 'h', 'l', 'close', 'v', 'oi']
                        elif len(df.columns) == 6:
                            df.columns = ['ts', 'o', 'h', 'l', 'close', 'v']
                            df['oi'] = 0
                        else:
                            df = df.iloc[:, :5]
                            df.columns = ['ts', 'o', 'h', 'l', 'close']
                            df['oi'] = 0
                            
                        # FIX: Handle String vs Int Timestamps
                        if isinstance(df['ts'].iloc[0], str):
                            df['timestamp'] = pd.to_datetime(df['ts'])
                        else:
                            df['timestamp'] = pd.to_datetime(df['ts'], unit='s')
                            
                        # Remove timezone for merging
                        df['timestamp'] = df['timestamp'].dt.tz_localize(None)
                        
                        df.set_index('timestamp', inplace=True)
                        df = df[~df.index.duplicated(keep='first')]
                        option_cache[sym] = df
                except Exception:
                    pass
        
        # Stitch
        for index, row in group.iterrows():
            ts = row['timestamp']
            atm = int(row['atm_strike'])
            
            def get_data(s, t):
                k = f"NSE-NIFTY-{active_fmt}-{s}-{t}"
                if k in option_cache and ts in option_cache[k].index:
                    d = option_cache[k].loc[ts]
                    return d['close'], d.get('oi', 0)
                return None, None

            atm_ce, atm_ce_oi = get_data(atm, "CE")
            atm_pe, atm_pe_oi = get_data(atm, "PE")
            itm_ce, itm_ce_oi = get_data(atm-50, "CE")
            itm_pe, itm_pe_oi = get_data(atm+50, "PE")
            
            rows_collected.append({
                "timestamp": ts,
                "nifty_open": row['open'],
                "nifty_high": row['high'],
                "nifty_low": row['low'],
                "nifty_close": row['close'],
                "expiry": expiry_str,
                "atm_strike": atm,
                "atm_ce_ltp": atm_ce, "atm_ce_oi": atm_ce_oi,
                "atm_pe_ltp": atm_pe, "atm_pe_oi": atm_pe_oi,
                "itm_ce_ltp": itm_ce, "itm_ce_oi": itm_ce_oi,
                "itm_pe_ltp": itm_pe, "itm_pe_oi": itm_pe_oi
            })
        
        option_cache.clear()

    # Save
    if rows_collected:
        final_df = pd.DataFrame(rows_collected)
        final_df.to_csv(OUTPUT_FILE, index=False)
        print(f"\n✅ PILOT SUCCESS! Saved {len(final_df)} rows to {OUTPUT_FILE}")
        print("First 5 Rows:")
        print(final_df[['timestamp', 'expiry', 'atm_ce_ltp', 'atm_ce_oi']].head())
    else:
        print("\n❌ PILOT FAILED: No rows collected.")

if __name__ == "__main__":
    run_pilot_test()

1. Loading Spot Data from Final_nifty_spot_1minute_fixed_trimmed.csv...

   Loaded 550600 rows.
2. Pilot Subset: 1502 rows (Jan 2020 & Recent).   Loaded 550600 rows.
2. Pilot Subset: 1502 rows (Jan 2020 & Recent).

Ready to Groww!

--- Starting Pilot Download ---

Processing Expiry: 02Jan20
Ready to Groww!

--- Starting Pilot Download ---

Processing Expiry: 02Jan20
   Using Date: 02Jan20 | Fetching 12 contracts...
   Using Date: 02Jan20 | Fetching 12 contracts...

Processing Expiry: 09Dec25

Processing Expiry: 09Dec25
   Using Date: 09Dec25 | Fetching 18 contracts...
   Using Date: 09Dec25 | Fetching 18 contracts...

Processing Expiry: 09Jan20

Processing Expiry: 09Jan20
   Using Date: 09Jan20 | Fetching 8 contracts...
   Using Date: 09Jan20 | Fetching 8 contracts...

✅ PILOT SUCCESS! Saved 1502 rows to test_pilot_output.csv
First 5 Rows:
            timestamp   expiry  atm_ce_ltp  atm_ce_oi
0 2020-01-01 09:15:00  02Jan20       40.15  2403225.0
1 2020-01-01 09:16:00  02Jan20       36.

In [12]:
import pandas as pd
import time
from datetime import timedelta
from growwapi import GrowwAPI

# --- CREDENTIALS ---
API_KEY = "eyJraWQiOiJaTUtjVXciLCJhbGciOiJFUzI1NiJ9.eyJleHAiOjI1NTM1Njg0NzYsImlhdCI6MTc2NTE2ODQ3NiwibmJmIjoxNzY1MTY4NDc2LCJzdWIiOiJ7XCJ0b2tlblJlZklkXCI6XCJhMTg3NDVhMy1hN2M1LTRlOTQtODE1MS1lZjUxZDQ5OGE2Y2RcIixcInZlbmRvckludGVncmF0aW9uS2V5XCI6XCJlMzFmZjIzYjA4NmI0MDZjODg3NGIyZjZkODQ5NTMxM1wiLFwidXNlckFjY291bnRJZFwiOlwiMDdmMDA0MGMtZTk4Zi00ZDNmLTk5Y2EtZDc1ZjBlYWU5M2NlXCIsXCJkZXZpY2VJZFwiOlwiZDMyMWIxMzUtZWQ5Mi01ZWJkLWJjMDUtZTY1NDY2OWRiMDM5XCIsXCJzZXNzaW9uSWRcIjpcIjBlOWMyYWZmLTM0NzktNDUyMi1iODE4LTczNTZlMzFkYmY1Y1wiLFwiYWRkaXRpb25hbERhdGFcIjpcIno1NC9NZzltdjE2WXdmb0gvS0EwYk1yOE5XVzhzdTNvZ080am1ZUzIwZEpSTkczdTlLa2pWZDNoWjU1ZStNZERhWXBOVi9UOUxIRmtQejFFQisybTdRPT1cIixcInJvbGVcIjpcImF1dGgtdG90cFwiLFwic291cmNlSXBBZGRyZXNzXCI6XCIyNDA5OjQwOTA6MTA4ZjpkYzA1OjNkMWQ6MWZmMDo1YWFjOjYwNTYsMTcyLjcxLjE5OC4xOSwzNS4yNDEuMjMuMTIzXCIsXCJ0d29GYUV4cGlyeVRzXCI6MjU1MzU2ODQ3NjQzNn0iLCJpc3MiOiJhcGV4LWF1dGgtcHJvZC1hcHAifQ.VuAMgqoC3e32gduObByNz97jFfG-ikXoREum26XPkvyMpj9JgCedXBI81jxGTPTrZD9i1wIL0s38LPd9vc9ApA"
API_SECRET = "xy0sbQ4r*!HN3&&UKc9vpwti4xx8PR)("

# --- CONFIGURATION ---
SPOT_FILE = "Final_nifty_spot_1minute_fixed_trimmed.csv"  # Your verified spot file
OUTPUT_FILE = "nifty_full_market_data_2020_2025.csv"      # The final result

def auth():
    try:
        token = GrowwAPI.get_access_token(api_key=API_KEY, secret=API_SECRET)
        return GrowwAPI(token)
    except Exception as e:
        print(f"CRITICAL: Auth failed. Error: {e}")
        exit()

# --- 1. SMART EXPIRY LOGIC (Handles 2025 Rule Change) ---
def get_target_expiry_date(date_obj):
    # Remove timezone info for safe comparison
    date_obj = date_obj.replace(tzinfo=None)
    
    # Logic: Thursday before Sep 1, 2025. Tuesday after.
    switch_date = pd.Timestamp("2025-09-01")
    target_day = 1 if date_obj >= switch_date else 3 # 1=Tue, 3=Thu
    
    days_ahead = target_day - date_obj.weekday()
    if days_ahead < 0: 
        days_ahead += 7
        
    return date_obj + timedelta(days=days_ahead)

def main():
    # 1. Load Data
    print(f"1. Loading Spot Data from {SPOT_FILE}...")
    try:
        df_spot = pd.read_csv(SPOT_FILE)
        
        # Robust Timestamp Parsing
        # (Handles both '2020-01-01' strings and Unix timestamps)
        if pd.api.types.is_numeric_dtype(df_spot['timestamp']):
            df_spot['timestamp'] = pd.to_datetime(df_spot['timestamp'], unit='s')
        else:
            df_spot['timestamp'] = pd.to_datetime(df_spot['timestamp'])
            
        # Normalize Timezone (Remove +05:30 for easier processing)
        df_spot['timestamp'] = df_spot['timestamp'].dt.tz_localize(None)
        
        df_spot.sort_values('timestamp', inplace=True)
        print(f"   Loaded {len(df_spot)} rows.")
        
    except FileNotFoundError:
        print("   Error: Spot file not found.")
        return

    # 2. Prepare Metadata
    print("2. Calculating Expiries & Strikes...")
    df_spot['atm_strike'] = (df_spot['close'] / 50).round() * 50
    df_spot['expiry_dt'] = df_spot['timestamp'].apply(get_target_expiry_date)
    # Groww Format: 09Jan20
    df_spot['expiry_str'] = df_spot['expiry_dt'].dt.strftime("%d%b%y")
    
    # 3. Start Download Loop
    groww = auth()
    option_cache = {}
    grouped = df_spot.groupby('expiry_str')
    
    print(f"\n--- Starting Full Download ({len(grouped)} Weeks) ---")
    
    # Check if we are resuming (append mode)
    file_exists = False
    try:
        pd.read_csv(OUTPUT_FILE, nrows=1)
        file_exists = True
        print("   (Appending to existing file...)")
    except FileNotFoundError:
        file_exists = False

    # Counters for progress
    processed_weeks = 0
    total_weeks = len(grouped)

    for expiry_str, group in grouped:
        processed_weeks += 1
        print(f"\n[{processed_weeks}/{total_weeks}] Processing Week: {expiry_str}")
        
        # A. Identify Strikes to Fetch (ATM, ITM, OTM)
        atm_strikes = group['atm_strike'].unique().astype(int)
        strikes_to_fetch = set()
        for k in atm_strikes:
            strikes_to_fetch.add(k)      # ATM
            strikes_to_fetch.add(k - 50) # ITM Call / OTM Put
            strikes_to_fetch.add(k + 50) # OTM Call / ITM Put
        
        # B. Smart Date Detection (Primary vs Backup)
        primary_dt = group['expiry_dt'].iloc[0]
        backup_dt = primary_dt - timedelta(days=1)
        fmt_primary = primary_dt.strftime("%d%b%y")
        fmt_backup = backup_dt.strftime("%d%b%y")
        
        active_fmt = fmt_primary
        
        # Quick Check: Does Primary Date have data?
        # We test the first strike to avoid wasting API calls on holidays
        test_sym = f"NSE-NIFTY-{fmt_primary}-{list(strikes_to_fetch)[0]}-CE"
        try:
            check_start = group['timestamp'].min().strftime("%Y-%m-%d %H:%M:%S")
            check_end = (group['timestamp'].min() + timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")
            resp = groww.get_historical_candles(
                exchange="NSE", segment="FNO", groww_symbol=test_sym,
                start_time=check_start, end_time=check_end, candle_interval="1minute"
            )
            # If empty list or None, assume holiday -> Switch to Backup
            if not (resp and "candles" in resp and len(resp["candles"]) > 0):
                active_fmt = fmt_backup
        except:
            active_fmt = fmt_backup
            
        print(f"   Date: {active_fmt} | Fetching {len(strikes_to_fetch)*2} contracts...")

        # C. Download Contracts
        start_dt_str = group['timestamp'].min().strftime("%Y-%m-%d %H:%M:%S")
        end_dt_str = group['timestamp'].max().strftime("%Y-%m-%d %H:%M:%S")
        
        for strike in strikes_to_fetch:
            for opt_type in ['CE', 'PE']:
                sym = f"NSE-NIFTY-{active_fmt}-{strike}-{opt_type}"
                if sym in option_cache: continue
                
                try:
                    resp = groww.get_historical_candles(
                        exchange="NSE", segment="FNO", groww_symbol=sym,
                        start_time=start_dt_str, end_time=end_dt_str, candle_interval="1minute"
                    )
                    
                    if resp and "candles" in resp and len(resp["candles"]) > 0:
                        df = pd.DataFrame(resp["candles"])
                        
                        # FIX: Handle Column Variations (6 vs 7 cols)
                        if len(df.columns) == 7:
                            df.columns = ['ts', 'o', 'h', 'l', 'close', 'v', 'oi']
                        elif len(df.columns) == 6:
                            df.columns = ['ts', 'o', 'h', 'l', 'close', 'v']
                            df['oi'] = 0 # Default if missing
                        else:
                            df = df.iloc[:, :5]
                            df.columns = ['ts', 'o', 'h', 'l', 'close']
                            df['oi'] = 0
                            
                        # FIX: Handle String vs Int Timestamps
                        if isinstance(df['ts'].iloc[0], str):
                            df['timestamp'] = pd.to_datetime(df['ts'])
                        else:
                            df['timestamp'] = pd.to_datetime(df['ts'], unit='s')
                        
                        # Remove timezone for clean merging
                        df['timestamp'] = df['timestamp'].dt.tz_localize(None)
                        
                        df.set_index('timestamp', inplace=True)
                        df = df[~df.index.duplicated(keep='first')]
                        option_cache[sym] = df
                except Exception:
                    pass
        
        # D. Stitch & Save
        rows_collected = []
        for index, row in group.iterrows():
            ts = row['timestamp']
            atm = int(row['atm_strike'])
            
            # Data Retrieval Helper
            def get_data(s, t):
                k = f"NSE-NIFTY-{active_fmt}-{s}-{t}"
                if k in option_cache and ts in option_cache[k].index:
                    d = option_cache[k].loc[ts]
                    return d['close'], d.get('oi', 0)
                return None, None

            # Fetch all needed data points
            atm_ce, atm_ce_oi = get_data(atm, "CE")
            atm_pe, atm_pe_oi = get_data(atm, "PE")
            itm_ce, itm_ce_oi = get_data(atm-50, "CE")
            itm_pe, itm_pe_oi = get_data(atm+50, "PE")
            
            rows_collected.append({
                "timestamp": ts,
                "nifty_open": row['open'],
                "nifty_high": row['high'],
                "nifty_low": row['low'],
                "nifty_close": row['close'],
                # "nifty_vol": row.get('volume', 0), # Optional
                "expiry": expiry_str,
                "atm_strike": atm,
                "atm_ce_ltp": atm_ce, "atm_ce_oi": atm_ce_oi,
                "atm_pe_ltp": atm_pe, "atm_pe_oi": atm_pe_oi,
                "itm_ce_ltp": itm_ce, "itm_ce_oi": itm_ce_oi,
                "itm_pe_ltp": itm_pe, "itm_pe_oi": itm_pe_oi
            })
        
        # Save Batch to Disk
        if rows_collected:
            final_df = pd.DataFrame(rows_collected)
            # Write header only if file didn't exist
            header_mode = not file_exists
            final_df.to_csv(OUTPUT_FILE, mode='a', header=header_mode, index=False)
            file_exists = True # Now it exists
            print(f"   ✅ Batch Saved ({len(final_df)} rows)")
        
        # E. Clear Memory
        option_cache.clear()

    print(f"\n🎉 SUCCESS! Full History Saved to {OUTPUT_FILE}")

if __name__ == "__main__":
    main()

1. Loading Spot Data from Final_nifty_spot_1minute_fixed_trimmed.csv...

   Loaded 550600 rows.
2. Calculating Expiries & Strikes...
   Loaded 550600 rows.
2. Calculating Expiries & Strikes...
Ready to Groww!

--- Starting Full Download (312 Weeks) ---Ready to Groww!

--- Starting Full Download (312 Weeks) ---


[1/312] Processing Week: 01Apr21

[1/312] Processing Week: 01Apr21
   Date: 01Apr21 | Fetching 26 contracts...
   Date: 01Apr21 | Fetching 26 contracts...
   ✅ Batch Saved (1501 rows)

[2/312] Processing Week: 01Aug24
   ✅ Batch Saved (1501 rows)

[2/312] Processing Week: 01Aug24
   Date: 01Aug24 | Fetching 30 contracts...
   Date: 01Aug24 | Fetching 30 contracts...
   ✅ Batch Saved (1881 rows)

[3/312] Processing Week: 01Dec22
   ✅ Batch Saved (1881 rows)

[3/312] Processing Week: 01Dec22
   Date: 01Dec22 | Fetching 24 contracts...
   Date: 01Dec22 | Fetching 24 contracts...
   ✅ Batch Saved (1879 rows)

[4/312] Processing Week: 01Feb24
   ✅ Batch Saved (1879 rows)

[4/312] Pr

In [13]:
import pandas as pd
import time
from datetime import timedelta
from growwapi import GrowwAPI

# --- CREDENTIALS ---
API_KEY = "eyJraWQiOiJaTUtjVXciLCJhbGciOiJFUzI1NiJ9.eyJleHAiOjI1NTM1Njg0NzYsImlhdCI6MTc2NTE2ODQ3NiwibmJmIjoxNzY1MTY4NDc2LCJzdWIiOiJ7XCJ0b2tlblJlZklkXCI6XCJhMTg3NDVhMy1hN2M1LTRlOTQtODE1MS1lZjUxZDQ5OGE2Y2RcIixcInZlbmRvckludGVncmF0aW9uS2V5XCI6XCJlMzFmZjIzYjA4NmI0MDZjODg3NGIyZjZkODQ5NTMxM1wiLFwidXNlckFjY291bnRJZFwiOlwiMDdmMDA0MGMtZTk4Zi00ZDNmLTk5Y2EtZDc1ZjBlYWU5M2NlXCIsXCJkZXZpY2VJZFwiOlwiZDMyMWIxMzUtZWQ5Mi01ZWJkLWJjMDUtZTY1NDY2OWRiMDM5XCIsXCJzZXNzaW9uSWRcIjpcIjBlOWMyYWZmLTM0NzktNDUyMi1iODE4LTczNTZlMzFkYmY1Y1wiLFwiYWRkaXRpb25hbERhdGFcIjpcIno1NC9NZzltdjE2WXdmb0gvS0EwYk1yOE5XVzhzdTNvZ080am1ZUzIwZEpSTkczdTlLa2pWZDNoWjU1ZStNZERhWXBOVi9UOUxIRmtQejFFQisybTdRPT1cIixcInJvbGVcIjpcImF1dGgtdG90cFwiLFwic291cmNlSXBBZGRyZXNzXCI6XCIyNDA5OjQwOTA6MTA4ZjpkYzA1OjNkMWQ6MWZmMDo1YWFjOjYwNTYsMTcyLjcxLjE5OC4xOSwzNS4yNDEuMjMuMTIzXCIsXCJ0d29GYUV4cGlyeVRzXCI6MjU1MzU2ODQ3NjQzNn0iLCJpc3MiOiJhcGV4LWF1dGgtcHJvZC1hcHAifQ.VuAMgqoC3e32gduObByNz97jFfG-ikXoREum26XPkvyMpj9JgCedXBI81jxGTPTrZD9i1wIL0s38LPd9vc9ApA"
API_SECRET = "xy0sbQ4r*!HN3&&UKc9vpwti4xx8PR)("

# --- CONFIGURATION ---
SPOT_FILE = "Final_nifty_spot_1minute_fixed_trimmed.csv"  # Your verified spot file
OUTPUT_FILE = "Order_nifty_full_market_data_2020_2025.csv"      # The final result

def auth():
    try:
        token = GrowwAPI.get_access_token(api_key=API_KEY, secret=API_SECRET)
        return GrowwAPI(token)
    except Exception as e:
        print(f"CRITICAL: Auth failed. Error: {e}")
        exit()

# --- 1. SMART EXPIRY LOGIC (Handles 2025 Rule Change) ---
def get_target_expiry_date(date_obj):
    # Remove timezone info for safe comparison
    date_obj = date_obj.replace(tzinfo=None)
    
    # Logic: Thursday before Sep 1, 2025. Tuesday after.
    switch_date = pd.Timestamp("2025-09-01")
    target_day = 1 if date_obj >= switch_date else 3 # 1=Tue, 3=Thu
    
    days_ahead = target_day - date_obj.weekday()
    if days_ahead < 0: 
        days_ahead += 7
        
    return date_obj + timedelta(days=days_ahead)

def main():
    # 1. Load Data
    print(f"1. Loading Spot Data from {SPOT_FILE}...")
    try:
        df_spot = pd.read_csv(SPOT_FILE)
        
        # Robust Timestamp Parsing
        if pd.api.types.is_numeric_dtype(df_spot['timestamp']):
            df_spot['timestamp'] = pd.to_datetime(df_spot['timestamp'], unit='s')
        else:
            df_spot['timestamp'] = pd.to_datetime(df_spot['timestamp'])
            
        # Normalize Timezone (Remove +05:30 for sorting)
        df_spot['timestamp'] = df_spot['timestamp'].dt.tz_localize(None)
        
        # CRITICAL: Sort Spot Data to ensure chronological order
        df_spot.sort_values('timestamp', inplace=True)
        print(f"   Loaded {len(df_spot)} rows.")
        
    except FileNotFoundError:
        print("   Error: Spot file not found.")
        return

    # 2. Prepare Metadata
    print("2. Calculating Expiries & Strikes...")
    df_spot['atm_strike'] = (df_spot['close'] / 50).round() * 50
    df_spot['expiry_dt'] = df_spot['timestamp'].apply(get_target_expiry_date)
    # Groww Format: 09Jan20
    df_spot['expiry_str'] = df_spot['expiry_dt'].dt.strftime("%d%b%y")
    
    # 3. Start Download Loop
    groww = auth()
    option_cache = {}
    
    # CRITICAL FIX: sort=False ensures we process Jan 2020 -> Feb 2020 ... -> Dec 2025
    grouped = df_spot.groupby('expiry_str', sort=False)
    
    print(f"\n--- Starting Full Download ({len(grouped)} Weeks) ---")
    
    # Check if we are resuming (append mode)
    file_exists = False
    try:
        pd.read_csv(OUTPUT_FILE, nrows=1)
        file_exists = True
        print("   (Appending to existing file...)")
    except FileNotFoundError:
        file_exists = False

    # Counters for progress
    processed_weeks = 0
    total_weeks = len(grouped)

    for expiry_str, group in grouped:
        processed_weeks += 1
        print(f"\n[{processed_weeks}/{total_weeks}] Processing Week: {expiry_str}")
        
        # A. Identify Strikes to Fetch (ATM, ITM, OTM)
        atm_strikes = group['atm_strike'].unique().astype(int)
        strikes_to_fetch = set()
        for k in atm_strikes:
            strikes_to_fetch.add(k)      # ATM
            strikes_to_fetch.add(k - 50) # ITM Call / OTM Put
            strikes_to_fetch.add(k + 50) # OTM Call / ITM Put
        
        # B. Smart Date Detection (Primary vs Backup)
        primary_dt = group['expiry_dt'].iloc[0]
        backup_dt = primary_dt - timedelta(days=1)
        fmt_primary = primary_dt.strftime("%d%b%y")
        fmt_backup = backup_dt.strftime("%d%b%y")
        
        active_fmt = fmt_primary
        
        # Quick Check: Does Primary Date have data?
        # We test the first strike to avoid wasting API calls on holidays
        test_sym = f"NSE-NIFTY-{fmt_primary}-{list(strikes_to_fetch)[0]}-CE"
        try:
            check_start = group['timestamp'].min().strftime("%Y-%m-%d %H:%M:%S")
            check_end = (group['timestamp'].min() + timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")
            resp = groww.get_historical_candles(
                exchange="NSE", segment="FNO", groww_symbol=test_sym,
                start_time=check_start, end_time=check_end, candle_interval="1minute"
            )
            # If empty list or None, assume holiday -> Switch to Backup
            if not (resp and "candles" in resp and len(resp["candles"]) > 0):
                active_fmt = fmt_backup
        except:
            active_fmt = fmt_backup
            
        print(f"   Date: {active_fmt} | Fetching {len(strikes_to_fetch)*2} contracts...")

        # C. Download Contracts
        start_dt_str = group['timestamp'].min().strftime("%Y-%m-%d %H:%M:%S")
        end_dt_str = group['timestamp'].max().strftime("%Y-%m-%d %H:%M:%S")
        
        for strike in strikes_to_fetch:
            for opt_type in ['CE', 'PE']:
                sym = f"NSE-NIFTY-{active_fmt}-{strike}-{opt_type}"
                if sym in option_cache: continue
                
                try:
                    resp = groww.get_historical_candles(
                        exchange="NSE", segment="FNO", groww_symbol=sym,
                        start_time=start_dt_str, end_time=end_dt_str, candle_interval="1minute"
                    )
                    
                    if resp and "candles" in resp and len(resp["candles"]) > 0:
                        df = pd.DataFrame(resp["candles"])
                        
                        # FIX: Handle Column Variations (6 vs 7 cols)
                        if len(df.columns) == 7:
                            df.columns = ['ts', 'o', 'h', 'l', 'close', 'v', 'oi']
                        elif len(df.columns) == 6:
                            df.columns = ['ts', 'o', 'h', 'l', 'close', 'v']
                            df['oi'] = 0 # Default if missing
                        else:
                            df = df.iloc[:, :5]
                            df.columns = ['ts', 'o', 'h', 'l', 'close']
                            df['oi'] = 0
                            
                        # FIX: Handle String vs Int Timestamps
                        if isinstance(df['ts'].iloc[0], str):
                            df['timestamp'] = pd.to_datetime(df['ts'])
                        else:
                            df['timestamp'] = pd.to_datetime(df['ts'], unit='s')
                        
                        # Remove timezone for clean merging
                        df['timestamp'] = df['timestamp'].dt.tz_localize(None)
                        
                        df.set_index('timestamp', inplace=True)
                        df = df[~df.index.duplicated(keep='first')]
                        option_cache[sym] = df
                except Exception:
                    pass
        
        # D. Stitch & Save
        rows_collected = []
        for index, row in group.iterrows():
            ts = row['timestamp']
            atm = int(row['atm_strike'])
            
            # Data Retrieval Helper
            def get_data(s, t):
                k = f"NSE-NIFTY-{active_fmt}-{s}-{t}"
                if k in option_cache and ts in option_cache[k].index:
                    d = option_cache[k].loc[ts]
                    return d['close'], d.get('oi', 0)
                return None, None

            # Fetch all needed data points
            atm_ce, atm_ce_oi = get_data(atm, "CE")
            atm_pe, atm_pe_oi = get_data(atm, "PE")
            itm_ce, itm_ce_oi = get_data(atm-50, "CE")
            itm_pe, itm_pe_oi = get_data(atm+50, "PE")
            
            rows_collected.append({
                "timestamp": ts,
                "nifty_open": row['open'],
                "nifty_high": row['high'],
                "nifty_low": row['low'],
                "nifty_close": row['close'],
                # "nifty_vol": row.get('volume', 0), # Optional
                "expiry": expiry_str,
                "atm_strike": atm,
                "atm_ce_ltp": atm_ce, "atm_ce_oi": atm_ce_oi,
                "atm_pe_ltp": atm_pe, "atm_pe_oi": atm_pe_oi,
                "itm_ce_ltp": itm_ce, "itm_ce_oi": itm_ce_oi,
                "itm_pe_ltp": itm_pe, "itm_pe_oi": itm_pe_oi
            })
        
        # Save Batch to Disk
        if rows_collected:
            final_df = pd.DataFrame(rows_collected)
            # Write header only if file didn't exist
            header_mode = not file_exists
            final_df.to_csv(OUTPUT_FILE, mode='a', header=header_mode, index=False)
            file_exists = True # Now it exists
            print(f"   ✅ Batch Saved ({len(final_df)} rows)")
        
        # E. Clear Memory
        option_cache.clear()

    print(f"\n🎉 SUCCESS! Full History Saved to {OUTPUT_FILE}")

if __name__ == "__main__":
    main()

1. Loading Spot Data from Final_nifty_spot_1minute_fixed_trimmed.csv...

   Loaded 550600 rows.
2. Calculating Expiries & Strikes...
   Loaded 550600 rows.
2. Calculating Expiries & Strikes...
Ready to Groww!
Ready to Groww!

--- Starting Full Download (312 Weeks) ---

[1/312] Processing Week: 02Jan20
--- Starting Full Download (312 Weeks) ---

[1/312] Processing Week: 02Jan20

   Date: 02Jan20 | Fetching 12 contracts...
   Date: 02Jan20 | Fetching 12 contracts...
   ✅ Batch Saved (750 rows)

[2/312] Processing Week: 09Jan20
   ✅ Batch Saved (750 rows)

[2/312] Processing Week: 09Jan20
   Date: 09Jan20 | Fetching 18 contracts...
   Date: 09Jan20 | Fetching 18 contracts...
   ✅ Batch Saved (1876 rows)

[3/312] Processing Week: 16Jan20
   ✅ Batch Saved (1876 rows)

[3/312] Processing Week: 16Jan20
   Date: 16Jan20 | Fetching 14 contracts...
   Date: 16Jan20 | Fetching 14 contracts...
   ✅ Batch Saved (1878 rows)

[4/312] Processing Week: 23Jan20
   ✅ Batch Saved (1878 rows)

[4/312] Proc

In [17]:
import pandas as pd
import numpy as np

# --- CONFIGURATION ---
INPUT_FILE = "Order_nifty_full_market_data_2020_2025.csv"
OUTPUT_FILE = "nifty_data_fixed_oi.csv"

def fix_data():
    print(f"1. Loading {INPUT_FILE}...")
    try:
        df = pd.read_csv(INPUT_FILE)
    except FileNotFoundError:
        print("Error: File not found.")
        return

    # Ensure Timestamp is clean
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    # --- STEP 1: FIX MISSING VALUES (GAPS) ---
    print("2. Filling missing values (Forward Fill)...")
    
    # These columns often have gaps because NSE updates them every 3 mins
    cols_to_fix = [
        'atm_ce_ltp', 'atm_ce_oi', 
        'atm_pe_ltp', 'atm_pe_oi',
        'itm_ce_ltp', 'itm_ce_oi',
        'itm_pe_ltp', 'itm_pe_oi'
    ]
    
    # Forward Fill: If data is missing at 09:16, use the value from 09:15
    df[cols_to_fix] = df[cols_to_fix].ffill()
    
    # Drop any rows that are STILL empty (e.g., the very first minute of 2020)
    df.dropna(subset=cols_to_fix, inplace=True)

    # --- STEP 2: FIX OI SCALING (NORMALIZATION) ---
    print("3. Normalizing Open Interest (Fixing 2025 Drop)...")
    
    # We create NEW columns called '_norm' (Normalized)
    # This solves the "Quantity vs Lots" issue by looking at RELATIVE changes
    oi_cols = ['atm_ce_oi', 'atm_pe_oi', 'itm_ce_oi', 'itm_pe_oi']
    
    for col in oi_cols:
        # Calculate Rolling Stats (2 Hours window)
        roll_mean = df[col].rolling(window=120).mean()
        roll_std = df[col].rolling(window=120).std()
        
        # Create Normalized Column (Z-Score)
        # 0 = Normal, +2 = Very High, -2 = Very Low
        # This makes 2024 and 2025 look exactly the same to the AI
        df[f'{col}_norm'] = (df[col] - roll_mean) / (roll_std + 1e-9)
        
        # We KEEP the original column too, just in case you want to see it.

    # Drop the first 120 rows where rolling mean is calculating
    df.dropna(inplace=True)

    # --- SAVE ---
    df.to_csv(OUTPUT_FILE, index=False)
    print("-" * 30)
    print(f"✅ SUCCESS! Cleaned file saved: {OUTPUT_FILE}")
    print(f"Total Rows: {len(df)}")
    print("New Columns Added: atm_ce_oi_norm, atm_pe_oi_norm, etc.")
    print("-" * 30)

if __name__ == "__main__":
    fix_data()

1. Loading Order_nifty_full_market_data_2020_2025.csv...

2. Filling missing values (Forward Fill)...
2. Filling missing values (Forward Fill)...
3. Normalizing Open Interest (Fixing 2025 Drop)...
3. Normalizing Open Interest (Fixing 2025 Drop)...
------------------------------
✅ SUCCESS! Cleaned file saved: nifty_data_fixed_oi.csv
Total Rows: 550481
New Columns Added: atm_ce_oi_norm, atm_pe_oi_norm, etc.
------------------------------
✅ SUCCESS! Cleaned file saved: nifty_data_fixed_oi.csv
Total Rows: 550481
New Columns Added: atm_ce_oi_norm, atm_pe_oi_norm, etc.
------------------------------
------------------------------


In [24]:
import pandas as pd

df = pd.read_csv("nifty_data_norm_oi.csv")

# Set option (this should ideally work)
pd.set_option('display.max_columns', None)

print("Shape:", df.shape)
print("\nColumns:\n", df.columns.tolist())

print("\nFirst 20 rows (Forced string representation):")
# Use .to_string() to force rendering without truncation
print(df.head(15).to_string()) 

print("\nlast 20 rows (Forced string representation):")
print(df.tail(15).to_string())

print("\nMissing values summary:")
print(df.isna().sum())


Shape: (550481, 19) (550481, 19)

Columns:
 

Columns:
 ['timestamp', 'nifty_open', 'nifty_high', 'nifty_low', 'nifty_close', 'expiry', 'atm_strike', 'atm_ce_ltp', 'atm_ce_oi', 'atm_pe_ltp', 'atm_pe_oi', 'itm_ce_ltp', 'itm_ce_oi', 'itm_pe_ltp', 'itm_pe_oi', 'atm_ce_oi_norm', 'atm_pe_oi_norm', 'itm_ce_oi_norm', 'itm_pe_oi_norm']

First 20 rows (Forced string representation):
['timestamp', 'nifty_open', 'nifty_high', 'nifty_low', 'nifty_close', 'expiry', 'atm_strike', 'atm_ce_ltp', 'atm_ce_oi', 'atm_pe_ltp', 'atm_pe_oi', 'itm_ce_ltp', 'itm_ce_oi', 'itm_pe_ltp', 'itm_pe_oi', 'atm_ce_oi_norm', 'atm_pe_oi_norm', 'itm_ce_oi_norm', 'itm_pe_oi_norm']

First 20 rows (Forced string representation):
              timestamp  nifty_open  nifty_high  nifty_low  nifty_close   expiry  atm_strike  atm_ce_ltp  atm_ce_oi  atm_pe_ltp  atm_pe_oi  itm_ce_ltp  itm_ce_oi  itm_pe_ltp  itm_pe_oi  atm_ce_oi_norm  atm_pe_oi_norm  itm_ce_oi_norm  itm_pe_oi_norm
0   2020-01-01 11:14:00    12184.15    12184.85   121

In [25]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
print("dtype:", df['timestamp'].dtype)
print("min/max:", df['timestamp'].min(), df['timestamp'].max())
print("tz info (should be None or with tz):", df['timestamp'].dt.tz)


dtype: datetime64[ns]
min/max: 2020-01-01 11:14:00 datetime64[ns]
min/max: 2020-01-01 11:14:00 2025-12-08 14:08:00
tz info (should be None or with tz): None 2025-12-08 14:08:00
tz info (should be None or with tz): None



In [26]:
df['ts'] = pd.to_datetime(df['timestamp']).dt.tz_localize(None)
df['time'] = df['ts'].dt.time
outside = df[~df['ts'].dt.strftime("%H:%M:%S").between("09:15:00","15:30:00")]
print("Rows outside 09:15-15:30:", len(outside))
print(outside.head())


Rows outside 09:15-15:30: 0
Empty DataFrame
Columns: [timestamp, nifty_open, nifty_high, nifty_low, nifty_close, expiry, atm_strike, atm_ce_ltp, atm_ce_oi, atm_pe_ltp, atm_pe_oi, itm_ce_ltp, itm_ce_oi, itm_pe_ltp, itm_pe_oi, atm_ce_oi_norm, atm_pe_oi_norm, itm_ce_oi_norm, itm_pe_oi_norm, ts, time]
Index: []
 0
Empty DataFrame
Columns: [timestamp, nifty_open, nifty_high, nifty_low, nifty_close, expiry, atm_strike, atm_ce_ltp, atm_ce_oi, atm_pe_ltp, atm_pe_oi, itm_ce_ltp, itm_ce_oi, itm_pe_ltp, itm_pe_oi, atm_ce_oi_norm, atm_pe_oi_norm, itm_ce_oi_norm, itm_pe_oi_norm, ts, time]
Index: []


In [27]:
dups = df[df.duplicated(subset='timestamp', keep=False)]
print("duplicate timestamp rows:", len(dups))

duplicate timestamp rows: 580
 580


In [28]:
df['ts'] = pd.to_datetime(df['timestamp'])
df = df.set_index('ts')
days = df.index.normalize().unique()
miss = {}
import pandas as pd
for d in days[:30]:   # check first 30 days quickly
    start = d + pd.Timedelta(hours=9, minutes=15)
    end   = d + pd.Timedelta(hours=15, minutes=30)
    full = pd.date_range(start=start, end=end, freq='1min')
    actual = df.loc[start:end].index.unique()
    miss_count = len(full) - len(actual)
    miss[str(d.date())] = miss_count
print("sample day missing minutes (first 30 days):")
print({k:v for k,v in list(miss.items())[:10]})


sample day missing minutes (first 30 days):
{'2020-01-01': 120, '2020-01-02': 1, '2020-01-03': 1, '2020-01-06': 0, '2020-01-07': 1, '2020-01-08': 1, '2020-01-09': 1, '2020-01-10': 1, '2020-01-13': 1, '2020-01-14': 0}

{'2020-01-01': 120, '2020-01-02': 1, '2020-01-03': 1, '2020-01-06': 0, '2020-01-07': 1, '2020-01-08': 1, '2020-01-09': 1, '2020-01-10': 1, '2020-01-13': 1, '2020-01-14': 0}


In [29]:
print("unique mod50:", sorted(df['atm_strike'].unique()[:10]))
print("any non-multiple of 50?", (df['atm_strike'] % 50).sum())


unique mod50:  [np.int64(11950), np.int64(12000), np.int64(12050), np.int64(12100), np.int64(12150), np.int64(12200), np.int64(12250), np.int64(12300), np.int64(12350), np.int64(12400)]
any non-multiple of 50? [np.int64(11950), np.int64(12000), np.int64(12050), np.int64(12100), np.int64(12150), np.int64(12200), np.int64(12250), np.int64(12300), np.int64(12350), np.int64(12400)]
any non-multiple of 50? 0
0


In [30]:
df['nifty_ret_1'] = df['nifty_close'].pct_change()
df['ce_ret_1'] = df['atm_ce_ltp'].pct_change()
df['pe_ret_1'] = df['atm_pe_ltp'].pct_change()
print("corr(spot,ce):", df['nifty_ret_1'].corr(df['ce_ret_1']))
print("corr(spot,pe):", df['nifty_ret_1'].corr(df['pe_ret_1']))


corr(spot,ce): 0.03158469935514146
corr(spot,pe): 0.03158469935514146
corr(spot,pe): -0.01680718375752286
 -0.01680718375752286


In [31]:
# clean_and_make_features.py
import pandas as pd
import numpy as np
from pathlib import Path

IN = "nifty_data_norm_oi.csv"
OUT_PARQUET = "master_ml_dataset.parquet"
GAP_REPORT = "gap_report.csv"
MARKET_TZ = "Asia/Kolkata"

df = pd.read_csv(IN)

# 1) parse timestamp and localize if naive
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
if df['timestamp'].dt.tz is None:
    df['timestamp'] = df['timestamp'].dt.tz_localize(MARKET_TZ)
else:
    df['timestamp'] = df['timestamp'].dt.tz_convert(MARKET_TZ)

# 2) sort & remove duplicates (keep last)
before = len(df)
df = df.sort_values('timestamp').drop_duplicates(subset='timestamp', keep='last').reset_index(drop=True)
dups_removed = before - len(df)
print(f"Duplicates removed: {dups_removed}")

# 3) trim strictly to market hours 09:15 - 15:30 (inclusive)
df = df.set_index('timestamp').sort_index()
df = df.between_time("09:15", "15:30")
df = df.reset_index()

# 4) per-day gap analysis and safe forward-fill for option columns
df['date'] = df['timestamp'].dt.date
option_cols = ['atm_ce_ltp','atm_pe_ltp','itm_ce_ltp','itm_pe_ltp','atm_ce_oi','atm_pe_oi','itm_ce_oi','itm_pe_oi']
report = []
filled_rows = 0
kept_rows = 0
rows_after = len(df)

# Group by date and handle each day individually
out_frames = []
for date_val, grp in df.groupby('date'):
    grp = grp.sort_values('timestamp').set_index('timestamp')
    start = pd.Timestamp(str(date_val) + " 09:15:00").tz_localize(MARKET_TZ)
    end   = pd.Timestamp(str(date_val) + " 15:30:00").tz_localize(MARKET_TZ)
    full_idx = pd.date_range(start=start, end=end, freq='1min', tz=MARKET_TZ)
    missing = len(full_idx) - len(grp)
    # If a huge gap (e.g., > 30 missing minutes), mark day as bad and skip (likely holiday/partial)
    if missing > 30:
        report.append({'date': str(date_val), 'missing_minutes': missing, 'action': 'DROP_DAY'})
        continue
    # Reindex and forward-fill option columns but limit fill to <=2 consecutive minutes
    re = grp.reindex(full_idx)
    # Count how many NaNs exist before fill
    na_before = re[option_cols].isna().sum().sum()
    # Forward fill with limit=2 for each option col
    for c in option_cols:
        if c in re.columns:
            re[c] = re[c].ffill(limit=2)
    na_after = re[option_cols].isna().sum().sum()
    filled = na_before - na_after
    filled_rows += filled
    # For spot columns (nifty_*), DO NOT fill — keep NaN if any (should be rare). We'll drop rows with spot NaN.
    re['date'] = date_val
    out_frames.append(re)
    report.append({'date': str(date_val), 'missing_minutes': missing, 'filled_option_values': int(filled), 'action': 'KEEP'})

# concat kept days
if out_frames:
    df_clean = pd.concat(out_frames).reset_index().rename(columns={'index':'timestamp'})
else:
    raise SystemExit("No valid trading days after gap filter - check data!")

# drop rows where spot is NaN (if any)
before_drop = len(df_clean)
df_clean = df_clean.dropna(subset=['nifty_close'])
dropped_for_spot = before_drop - len(df_clean)

# 5) rolling OI z-score per expiry (60-min window) - safer normalization
df_clean['atm_ce_oi'] = pd.to_numeric(df_clean['atm_ce_oi'], errors='coerce')
df_clean['atm_pe_oi'] = pd.to_numeric(df_clean['atm_pe_oi'], errors='coerce')
df_clean['itm_ce_oi'] = pd.to_numeric(df_clean['itm_ce_oi'], errors='coerce')
df_clean['itm_pe_oi'] = pd.to_numeric(df_clean['itm_pe_oi'], errors='coerce')

def rolling_z(s, w=60):
    mu = s.rolling(window=w, min_periods=1).mean()
    sd = s.rolling(window=w, min_periods=1).std(ddof=0).replace(0, np.nan)
    return (s - mu) / (sd + 1e-9)

for col in ['atm_ce_oi','atm_pe_oi','itm_ce_oi','itm_pe_oi']:
    df_clean[col + '_z60'] = df_clean.groupby('expiry')[col].transform(lambda s: rolling_z(s, w=60))

# 6) core features
df_clean['nifty_ret_1'] = df_clean['nifty_close'].pct_change().fillna(0)
df_clean['nifty_ret_3'] = df_clean['nifty_close'].pct_change(3).fillna(0)
df_clean['ce_ret_1'] = df_clean['atm_ce_ltp'].pct_change().fillna(0)
df_clean['pe_ret_1'] = df_clean['atm_pe_ltp'].pct_change().fillna(0)
df_clean['ce_pe_ratio'] = df_clean['atm_ce_ltp'] / (df_clean['atm_pe_ltp'] + 1e-9)
df_clean['ce_minus_pe'] = df_clean['atm_ce_ltp'] - df_clean['atm_pe_ltp']
# seasonality
df_clean['minute_of_day'] = df_clean['timestamp'].dt.hour*60 + df_clean['timestamp'].dt.minute
df_clean['sin_min'] = np.sin(2*np.pi*df_clean['minute_of_day']/390)
df_clean['cos_min'] = np.cos(2*np.pi*df_clean['minute_of_day']/390)

# 7) quick stats
print("Rows before cleaning:", before)
print("Rows after dedupe & trim:", rows_after)
print("Rows dropped for heavy-gap days:", len(report) - sum(1 for r in report if r['action']=='KEEP'))
print("Option values forward-filled total:", filled_rows)
print("Rows dropped due to missing spot:", dropped_for_spot)
print("Final rows:", len(df_clean))

# 8) save outputs
df_clean.to_parquet(OUT_PARQUET, index=False)
pd.DataFrame(report).to_csv(GAP_REPORT, index=False)
print("Saved:", OUT_PARQUET, GAP_REPORT)


Duplicates removed: 290

Rows before cleaning: 550481
Rows before cleaning: 550481
Rows after dedupe & trim: 550191
Rows after dedupe & trim: 550191
Rows dropped for heavy-gap days: 10
Rows dropped for heavy-gap days: 10
Option values forward-filled total: 4944Option values forward-filled total: 4944
Rows dropped due to missing spot: 618
Rows dropped due to missing spot: 618
Final rows: 547966
Final rows: 547966

Saved: master_ml_dataset.parquet Saved: master_ml_dataset.parquet gap_report.csv
gap_report.csv


In [34]:
import pandas as pd
df = pd.read_parquet("master_labeled.parquet")

print("Total rows:", len(df))
print("NaN summary:")
print(df.isna().sum().sort_values().tail(40))


Total rows: 547963
NaN summary:
 547963
NaN summary:
ce_pe_ratio_lag1             1
atm_itm_spread_ce_lag1       1
atm_itm_spread_ce_lag2       2
ce_pe_ratio_lag2             2
itm_pe_ret_1_lag1            2
ce_ret_1_lag1                2
nifty_ret_1_lag1             2
atm_itm_spread_pe_lag2       2
ce_minus_pe_lag2             2
pe_ret_1_lag1                2
itm_ce_ret_1_lag1            2
ce_minus_pe_lag3             3
nifty_ret_1_lag2             3
ce_ret_1_lag2                3
atm_itm_spread_pe_lag3       3
atm_itm_spread_ce_lag3       3
itm_pe_ret_1_lag2            3
pe_ret_1_lag2                3
itm_ce_ret_1_lag2            3
ce_pe_ratio_lag3             3
nifty_ret_1_lag3             4
ce_ret_1_lag3                4
itm_pe_ret_1_lag3            4
itm_ce_ret_1_lag3            4
nifty_ret_3_lag1             4
pe_ret_1_lag3                4
nifty_ret_3_lag2             5
nifty_ret_3_lag3             6
itm_ce_oi_z60_lag1        2829
itm_ce_oi_z60_lag2        2830
itm_ce_oi_z60_lag

In [35]:
df[['nifty_close', 'future_ret_1m', 'target_dir_1m', 
    'target_up_small_1m', 'future_ret_3m', 'target_up_med_3m']].sample(10)


,nifty_close,future_ret_1m,target_dir_1m,target_up_small_1m,future_ret_3m,target_up_med_3m
472781,23594.45,0.000040,1,0,-0.000074,0
219459,16262.15,0.000852,1,1,0.000658,1
406637,22945.65,-0.000105,0,0,0.000610,1
398673,22393.30,0.000174,1,0,0.000295,0
359313,19738.75,-0.000494,0,0,0.000114,0
362425,20277.55,0.000234,1,1,0.000128,0
88551,13513.50,0.000155,1,0,-0.000958,0
232112,16022.35,-0.000122,0,0,-0.000278,0
208930,18013.85,-0.000411,0,0,-0.000083,0
145062,15750.35,0.000000,0,0,-0.000108,0


In [36]:
df[['nifty_ret_1','ce_ret_1','pe_ret_1','ce_pe_ratio','atm_ce_oi_z60']].corr()


,nifty_ret_1,ce_ret_1,pe_ret_1,ce_pe_ratio,atm_ce_oi_z60
nifty_ret_1,1.000000,0.033924,-0.024764,-0.010960,0.048224
ce_ret_1,0.033924,1.000000,0.037737,0.022872,-0.005034
pe_ret_1,-0.024764,0.037737,1.000000,-0.001338,-0.002501
ce_pe_ratio,-0.010960,0.022872,-0.001338,1.000000,-0.043752
atm_ce_oi_z60,0.048224,-0.005034,-0.002501,-0.043752,1.000000


In [37]:
[f for f in df.columns if 'future' in f]


['future_close_1m', 'future_ret_1m', 'future_close_3m', 'future_ret_3m']

In [38]:
import pandas as pd
sr = pd.read_csv("backtest_outputs/sweep_results.csv")
print(sr.shape)
display(sr.head(20))
display(sr.describe(include='all'))


(266, 10)



,target_min,threshold,sl_frac,tp_frac,trades,wins,precision,trades_per_day,avg_pnl,total_pnl
0,1,0.90,NaN,NaN,0,0,0.0,0.000000,0.000000,0.000000
1,1,0.88,NaN,NaN,0,0,0.0,0.000000,0.000000,0.000000
2,1,0.86,NaN,NaN,0,0,0.0,0.000000,0.000000,0.000000
3,1,0.84,NaN,NaN,0,0,0.0,0.000000,0.000000,0.000000
4,1,0.82,NaN,NaN,0,0,0.0,0.000000,0.000000,0.000000
5,1,0.80,NaN,NaN,0,0,0.0,0.000000,0.000000,0.000000
6,1,0.78,NaN,NaN,0,0,0.0,0.000000,0.000000,0.000000
7,1,0.76,0.0003,NaN,3,0,0.0,0.001386,-43.222381,-129.667142
8,1,0.76,0.0003,0.0006,3,0,0.0,0.001386,-43.222381,-129.667142
9,1,0.76,0.0003,0.0010,3,0,0.0,0.001386,-43.222381,-129.667142


,target_min,threshold,sl_frac,tp_frac,trades,wins,precision,trades_per_day,avg_pnl,total_pnl
count,266.000000,266.000000,168.0000,168.000000,266.000000,266.000000,266.000000,266.000000,266.000000,2.660000e+02
mean,2.000000,0.641053,0.0004,0.000800,29391.157895,6.203008,0.000279,13.575593,-39.967257,-1.837485e+05
std,1.001885,0.092048,0.0001,0.000201,60111.417458,29.334572,0.000956,27.765089,9.597999,3.241909e+05
min,1.000000,0.500000,0.0003,0.000600,0.000000,0.000000,0.000000,0.000000,-47.642777,-1.237566e+06
25%,1.000000,0.560000,0.0003,0.000600,32.000000,0.000000,0.000000,0.014781,-43.022959,-2.074417e+05
50%,2.000000,0.640000,0.0004,0.000800,664.000000,0.000000,0.000000,0.306697,-42.401339,-2.366894e+04
75%,3.000000,0.720000,0.0005,0.001000,24095.000000,0.000000,0.000000,11.129330,-41.548805,-1.378062e+03
max,3.000000,0.900000,0.0005,0.001000,218641.000000,234.000000,0.005780,100.988915,0.000000,0.000000e+00


In [39]:
import pandas as pd
preds = pd.read_csv("path/to/preds.csv", parse_dates=['timestamp']) # rename to your file/col
master = pd.read_csv("path/to/master.csv", parse_dates=['timestamp'])
# quick sanity
print("preds:", preds.shape, "master:", master.shape)
print("preds ts min/max:", preds['timestamp'].min(), preds['timestamp'].max())
print("master ts min/max:", master['timestamp'].min(), master['timestamp'].max())

# how many preds matched during merge (if you have a merge_count column or compute join)
merged = preds.merge(master, on='timestamp', how='left', indicator=True)
print(merged['_merge'].value_counts())
# show rows that didn't match
print(merged[merged['_merge']!='both'].head(20))


FileNotFoundError: [Errno 2] No such file or directory: 'path/to/preds.csv'